# Transfer learning with tensorflow - feature extraction

## Downloading and familiarizing with the data

In [1]:
# Get data (10% of 10 food classes from Food101)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2025-01-27 19:15:54--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.27, 142.250.65.123, 172.217.15.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   193MB/s    in 0.8s    

2025-01-27 19:15:55 (193 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [2]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent

## Creating data loaders

In [3]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks

In [4]:
# Create tensorboard callback
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## Creating models using tensorflow hub

In [5]:
# Let's compare 2 models
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4'
efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [6]:
# import dependencies
# !pip install tensorflow==2.15.0 tensorflow-hub==0.13.0 keras==2.15.0 --force-reinstall

In [7]:
# import dependencies
# !pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# Let's make a create_model() function
def create_model(model_url, num_classes=10):
  # Download pretrained model
  feature_extractor_layer = hub.KerasLayer(model_url,
                                          trainable=False,
                                          name='feature_extraction_layer',
                                          input_shape=IMAGE_SHAPE+(3,))
  # Create our own model
  # model = tf.keras.Sequential([
  #   feature_extractor_layer,
  #   layers.Dense(num_classes, activation='softmax', name='output_layer')
  # ])
  model = tf.keras.Sequential()
  model.add(feature_extractor_layer)
  model.add(tf.keras.layers.Dense(num_classes, activation=tf.keras.activations.softmax, name='output_layer'))

  return model

### Resnet TensorFlow Hub Feature extraction model

In [9]:
# Create resnet model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

In [10]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 2048)              23564800  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [11]:
# Compile model
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [12]:
# Fit the model
# resnet_history = resnet_model.fit(train_data_10_percent,
#                                    epochs=5,
#                                    steps_per_epoch=len(train_data_10_percent),
#                                   validation_data=test_data,
#                                   validation_steps=len(test_data),
#                                   callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
#                                                                          experiment_name='resnet')])

In [13]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']
  epochs = range(len(history.history['loss']))

  #
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()
  plt.figure()

  #
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();


In [14]:
# plot_loss_curves(resnet_history);

## Create and test and EfficientNerB0 model

In [15]:
efficient_model = create_model(model_url=efficientnet_url, num_classes=train_data_10_percent.num_classes)

efficient_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])

In [16]:
# efficient_history = efficient_model.fit(train_data_10_percent,
#                                        epochs=5,
#                                        steps_per_epoch=len(train_data_10_percent),
#                                        validation_data=test_data,
#                                        validation_steps=len(test_data),
#                                        callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
#                                                                               experiment_name='efficientnet')])

In [17]:
plot_loss_curves(efficient_history);

NameError: name 'efficient_history' is not defined

In [18]:
efficient_model.layers[0].trainable

False

In [ ]:
# Upload TesorBoard dev records
!tensorboard dev u;load --logdir ./tensorflow_hub/ \
  --name "efficientnet vs resnet" \
  --description "comparing 2 image classification models" \
  --one_shot